---
# Create runoff to ocean mapping files for CESM3 LGM
- Author: Jiang Zhu (jiangzhu@ucar.edu)
- You could also run this in shell environment
- Be sure to use enough memory (I used 40 GB on Casper)
- This step depends on the LGM SCRIP file from create_mom6_bathy.ipynb
- Tools used
  - [runoff_to_ocn](https://github.com/ESMCI/cime/blob/5eff840b3eb6243b97d65a1989cedc5ebcbcd3c7/tools/mapping/gen_mapping_files/runoff_to_ocn/README)

In [1]:
import subprocess
import datetime

import warnings
warnings.filterwarnings('ignore')

---
# Input files and settings for generating mapping files

In [2]:
ocn_SCRIP = '/glade/work/jiangzhu/data/inputdata/mom/tx2_3v2/tx2_3v2_SCRIP_230415_21ka_250922.nc'

base_mapping = {
    'src_gridname': 'r05',
    'src_gridtype': 'rtm',
    'src_grid': '/glade/campaign/cesm/cesmdata/inputdata/lnd/clm2/rtmdata/rdirc.05.061026',
}

mapping_files = {
    'r2o_liq': {**base_mapping, 'eFold': 250000.0, 'rMax': 250000.0},
    'r2o_ice': {**base_mapping, 'eFold': 1000000.0, 'rMax': 1000000.0},
}

---
# Output directory (output file names will be derived automatically)

In [3]:
today = datetime.date.today().strftime("%Y%m%d")
print(today)

work_dir = '/glade/work/jiangzhu/data/inputdata/cesm3_21ka'

20250922


---
# Tool used

In [4]:
runoff_map_exe = '/glade/campaign/cesm/development/palwg/scripts/setup/runoff_to_ocn/runoff_map'

---
# River mapping with multiple settings in eFold & rMax

In [5]:
%%time

for key in mapping_files.keys():
    print(key)

    src_gridname = mapping_files[key]['src_gridname']
    src_gridtype = mapping_files[key]['src_gridtype']
    src_grid = mapping_files[key]['src_grid']
    eFold = mapping_files[key]['eFold']
    rMax = mapping_files[key]['rMax']

    eFold_km = f"{eFold/1e3:.0f}"
    rMax_km = f"{rMax/1e3:.0f}"

    file_nn     = f'{work_dir}/map_{src_gridname}_to_tx2_3v2_21ka_nn_e{eFold_km}r{rMax_km}_{today}.nc'
    file_smooth = f'{work_dir}/map_{src_gridname}_to_tx2_3v2_21ka_sm_e{eFold_km}r{rMax_km}_{today}.nc'
    file_new    = f'{work_dir}/map_{src_gridname}_to_tx2_3v2_21ka_nnsm_e{eFold_km}r{rMax_km}_{today}.nc'

    with open(f"{work_dir}/map_{src_gridname}_to_tx2_3v2_21ka_e{eFold_km}r{rMax_km}.nml", "w") as f:

        f.write('&input_nml\n')
        f.write(f"gridtype     = '{src_gridtype}'\n")
        f.write(f"file_roff    = '{src_grid}'\n")
        f.write(f"file_ocn     = '{ocn_SCRIP}'\n")
        f.write(f"file_nn      = '{file_nn}'\n")
        f.write(f"file_smooth  = '{file_smooth}'\n")
        f.write(f"file_new     = '{file_new}'\n")
        f.write(f"title        = 'runoff map: {src_gridname} -> tx2_3v2 21ka, nearest neighbor and smoothed'\n")
        f.write(f"eFold        = {eFold}\n")
        f.write(f"rMax         = {rMax}\n")
        f.write(f"step1        = .true.\n")
        f.write(f"step2        = .true.\n")
        f.write(f"step3        = .true.\n")
        f.write(f"/\n")

    cmd = f"{runoff_map_exe} < {work_dir}/map_{src_gridname}_to_tx2_3v2_21ka_e{eFold_km}r{rMax_km}.nml"  
    print(cmd)
    subprocess.run(cmd, shell=True)

r2o_liq
/glade/campaign/cesm/development/palwg/scripts/setup/runoff_to_ocn/runoff_map < /glade/work/jiangzhu/data/inputdata/cesm3_21ka/map_r05_to_tx2_3v2_21ka_e250r250.nml
(main) -------------------------------------------------------------------------
(main) correct/smooth/sort runoff -> ocean map
(main) -------------------------------------------------------------------------
(main) -------------------------------------------------------------------------
(main) Step 0:  read input namelist data
(main) -------------------------------------------------------------------------
(main) Namelist values...
(main)    gridtype                       = rtm
(main)    file_roff                      = /glade/campaign/cesm/cesmdata/inputdata/lnd/clm2/rtmdata/rdirc.05.061026
(main)    file_ocn (global)              = /glade/work/jiangzhu/data/inputdata/mom/tx2_3v2/tx2_3v2_SCRIP_230415_21ka_250922.nc
(main)    file_ocn (coast)               = /glade/work/jiangzhu/data/inputdata/mom/tx2_3v2/tx2_3v2_S